In [2]:
%pylab inline
from __future__ import print_function, division
from IPython.core.display import Image
from IPython.core.display import display_html
import my_dsp_lib_v2 as dsp
#imgpath = '/user_home/w_Chipmuenk/DSV/' # Wakari operation
imgpath = 'D:/Daten/HM/dsvFPGA/python/IPy_Notebooks/dsvFPGA/' # local operation
FigSize = (7,7) # size of figures in inches

Populating the interactive namespace from numpy and matplotlib


SyntaxError: Missing parentheses in call to 'print' (my_dsp_lib_v2.py, line 349)

<!-- example for static HTML image embedded in MarkDown
![Menschen und Python](http://imgs.xkcd.com/comics/reassuring.png) 
MarkDown allows no alignment and scaling, but HTML/CSS does: -->

<p style="text-align:center; font-weight:bold">
<img src="http://imgs.xkcd.com/comics/reassuring.png" />
<br />Python macht glücklich ...
</p>



# Spektrale Fehlerformung
Wie in den vorigen Kapiteln gezeigt wurde, kann man den nicht-linearen Prozess der (Re-)Quantisierung beschreiben durch Addition des Quantisierungsfehlers $\epsilon(t)$ bzw. $\epsilon[n]$ (für zeitdiskrete Signale) zum Signal. Wenn der Quantisierungsfehler bestimmte Bedingungen erfüllt (stationärer Prozess mit unkorrelierten Werten), ist eine einfache lineare Beschreibung im Frequenzbereich möglich, da das Spektrum von $\epsilon[n]$ dann weiß ist.

In diesem Kapitel soll gezeigt werden, dass bei Systemen mit Überabtastung der Quantisierer verbessert werden kann, indem man Rauschen aus dem Nutzband zu höheren Frequenzen schiebt ("*noise shaping*" oder "*error spectral shaping*"). Dies wird erreicht durch Rückkoppeln und Filtern des Quantisierungsfehlers.

<img src="./Linearized_Quantizer_2.png" width="600">

## Zeitdiskrete Systeme
Aufgrund der Multiplikationen mit den Koeffizienten nimmt die Wortlänge in digitalen Filtern immer zu. Bei rekursiven Filtern muss daher die Wortlänge reduziert (= requantisiert) werden bevor die Werte des rekursiven Pfads mit den Eingangswerten kombiniert werden.

In [ ]:
NFFT = 1024; x = linspace(0,6*pi,NFFT);  a = 1.5*sin(2*x)
q_obj = (1,2,'round','sat') # try 'floor' ; 'wrap'
aq, N_over = dsp.fixed(q_obj, a)
fig1=figure(figsize=(FigSize[0]*2,FigSize[1])); clf()
ax1 = fig1.add_subplot(2,2,1); ax1.plot(x,aq)
print('Number of overflows = ', N_over) 
ax1.plot(x, a, 'r', label = r'$a(x)$'); grid(True)
ax1.step(x, aq, where = 'post', label = r'$a_Q(x)$')
ax1.plot(x, a-aq, label = r'$a(x) - a_Q(x)$')
ax1.legend(fontsize = 14); ax1.set_xlabel(r'$x \rightarrow$')
ax2 = fig1.add_subplot(2,2,2);
A = fft.fft(a)/NFFT; f = fftfreq(NFFT, 1e-3)
AQ = fft.fft(aq)/NFFT; 
ax2.plot(f[0:NFFT/2.0 - 1],20*log10(2*abs(A[0:NFFT/2.0-1])), label = r'$A(f)$'); grid(True)
ax2.plot(f[0:NFFT/2.0 - 1],20*log10(2*abs(AQ[0:NFFT/2.0-1])), label = r'$A_Q(f)$')
ax2.legend(fontsize = 14); ax2.set_xlabel(r'$f \rightarrow$')
ax2.set_ylim(-100,10)

In [ ]:
x = linspace(0,2*pi,100)
a = sin(2*x)
q_obj = (0,2,'round','sat')
aq, N_over = dsp.fixed(q_obj, a)
plt.plot(x,aq)
print 'Number of overflows = ', N_over 
clf()
plot (x,aq, x, a-aq); grid(True)
legend(('Q<a>', 'a - Q<a>'))

## Sigma-Delta ADCs
$\Sigma \Delta$ ADCs enthalten sehr grobe Quantisierer (oft nur 1 Bit!), erzielen aber durch Noise Shaping *effective number of bits* (ENOB) von bis zu 24 Bits.


<p style="text-align:center; font-weight:bold">
<img src="http://upload.wikimedia.org/wikipedia/commons/f/f9/Fig._1b.svg" />
<br />Sigma-Delta Modulator erster Ordnung [wikipedia.org]
</p>